In [ ]:
from __future__ import print_function
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, gridspec
import matplotlib as mpl

import os
def load_data_dir(DIR):
    data = []
    error_files = []
    for fname in sorted(os.listdir(DIR)):
        if fname.startswith('run'):
            ffname = DIR+'/'+fname
            with open(ffname) as f:
                for l in f.readlines():
                    if l[0]=='#' and 'num_memes' in l:
                        try:
                            d = dict(e.split('=', 1) for e in l[1:].strip().split(', '))
                            d['filename'] = fname
                            df = pd.read_csv(ffname, comment='#', sep=' ')
                            if not len(df):
                                raise
                            data.append((d,df))
                        except:
                            error_files.append(ffname)
    return data, error_files

In [ ]:
basedir = 'bm_sfi_scaling002'
DIR, COL  = (basedir+'/agents', 'num_agents')
#DIR, COL  = (basedir+'/lexesmemes', 'num_memes')
#DIR, COL  = (basedir+'/lexesmemesagents', 'num_agents')
DO_SAVE  = True # whether to save pdf output
MAX_DRIFT_PLOT = 0.005
IS_CONVERGED_CUTOFF = 0.75

max_step = 0
data, error_files = load_data_dir(DIR)

mutators = set(cfg['mutator_class'] for cfg, _ in data)
colvals  = set(float(cfg[COL])      for cfg, _ in data)

print("Loaded %d files" % len(data))
if len(error_files):
    print("Errors processing these files:")
    for ffname in error_files:
        print("  %s" % ffname)


In [ ]:
for m in sorted(mutators):
    plt.figure(figsize=(12,5))
    for v in sorted(colvals):
        for (cfg, df) in data:
            if cfg['mutator_class'] != m or float(cfg[COL]) != v:
                continue
            plt.subplot(1,2,1)
            plt.plot(df.Step, df.Comprehension, label=cfg[COL])
            plt.semilogx()
            plt.title(m)
            plt.legend()
            plt.subplot(1,2,2)
            rescaled = df.Step/(v**1.25)
            plt.plot(rescaled, df.Comprehension, label=cfg[COL])
            if v==21:
                plt.xlim([0, 2*np.max(rescaled)])
            plt.semilogx()
            plt.title(m)
    